In [1]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Concatenate, Dropout, Dense, BatchNormalization, Dot
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from tensorflow import keras
import numpy as np

In [2]:
!git clone https://github.com/SyedMa3/ncf.git
%cd ncf

Cloning into 'ncf'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 110 (delta 56), reused 54 (delta 21), pack-reused 0
Receiving objects: 100% (110/110), 14.31 MiB | 8.14 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/ncf


In [3]:
def get_model(num_users, num_items, latent_dim):

  user_input = Input(shape=(1,), dtype = 'int32', name = 'user_input')
  item_input = Input(shape=(1,), dtype = 'int32', name = 'item_input')

  GMF_user_embedding = Embedding(num_users, latent_dim, name='gmf_user_embedding', input_length=1)
  GMF_item_embedding = Embedding(num_items, latent_dim, name='gmf_item_embedding', input_length=1)

  user_latent = Flatten()(GMF_user_embedding(user_input))
  item_latent = Flatten()(GMF_item_embedding(item_input))

  mf_pred = Dot(axes=1)([user_latent, item_latent])

  mlp_user = Embedding(num_users+1, 10, name='user_embedding')(user_input)
  mlp_item = Embedding(num_items+1, 10, name='item_embedding')(item_input)

  mlp_user = Flatten()(mlp_user)
  mlp_item = Flatten()(mlp_item)

  concat = Concatenate()([mlp_item, mlp_user])
  concat = Dropout(0.2)(concat)

  x = Dense(32, activation='relu')(concat)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Dense(16, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Dense(8, activation='relu')(x)

  merged = Concatenate()([mf_pred, x])

  prediction = Dense(1, activation='sigmoid', name='prediction')(merged)

  model = Model([user_input, item_input], prediction)
  # model.summary()
  return model

In [4]:
import imp
dataset = imp.new_module('dataset')
exec(open("./dataset.py").read(), dataset.__dict__)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [5]:
a = np.array(dataset.users).reshape(-1,1)
b = np.array(dataset.items).reshape(-1,1)
c = np.array(dataset.labels).reshape(-1,1)

In [6]:
m = get_model(dataset.num_users,dataset.num_movies,10)
m.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy')
m.summary()
m.fit([a,b], c, batch_size=256, epochs=10)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        1936100     ['item_input[0][0]']             
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        6110        ['user_input[0][0]']             
                                                                                              

In [7]:
m.save('model.h5')